In [21]:
import pandas as pd
import requests
import json
from datetime import datetime
import urllib
from sqlalchemy import create_engine, types
from sqlalchemy.pool import NullPool
from hubspot.crm.contacts import SimplePublicObjectInput, ApiException
from hubspot.auth.oauth import ApiException
from validate_email import validate_email
import hubspot
import os
from dotenv import load_dotenv

load_dotenv()
DATABASE_CONNECTION_URI = os.environ["DB_URL"]
ACCESS_TOKEN = os.environ["ACCESS_TOKEN"]
client_id = os.environ["CLIENT_ID"]
client_secret = os.environ["CLIENT_SECRET"]

# create a connection to the database
engine = create_engine(DATABASE_CONNECTION_URI)

In [22]:
class HubspotAPI:
    def __init__(self):
        # API KEY
        self.access_token = ACCESS_TOKEN
        self.client = hubspot.Client.create(access_token = self.access_token)
        self.max_results = 1000000

    def raw_export_contacts(self, properties_dict):
        # Assistant Variables
        results = []
        after = 0

        while str(after).isnumeric() and len(results) < self.max_results:
            try:
                api_response = self.client.crm.contacts.basic_api.get_page(
                    limit=100,
                    after=after,
                    properties=list(properties_dict.keys()),
                    associations=["deals", "companies"],
                    archived=False,
                )
                api_response = api_response.to_dict()
                results.extend(api_response['results'])
                print("Hubspot Contacts Export has gathered " + str(len(results)) + " Contacts")
                try:
                    after = api_response['paging']['next']['after']
                except:
                    after = api_response['paging']
            except ApiException as e:
                print("Exception when calling basic_api->get_page: %s\n" % e)
        property_results = []
        for result in results:
            property_results.append(result["properties"])
        df = pd.DataFrame(property_results)
        df.to_csv("raw_hubspot_contacts.csv", encoding="latin-1", index=False, errors='ignore')
        assoc_results = []
        itterator = results.copy()
        for result in itterator:
            try:
                lists_to_append = result["associations"]["deals"]["results"]
                to_append = []
                for i in lists_to_append:
                    dict_to_append = {}
                    dict_to_append["contact_id"] = result["id"]
                    dict_to_append["deal_id"] = i["id"]
                    to_append.append(dict_to_append)
                assoc_results.append(to_append)
            except:
                assoc_results.append([{"contact_id": result["id"], "deal_id": None}])
        contacts_id = []
        deal_id = []
        for index, item in enumerate(assoc_results):
            i = 0
            for a, b in item:
                contacts_id.append(item[i][a])
                deal_id.append(item[i][b])
                i += 1
        df = pd.DataFrame({"contact_id": contacts_id, "deals_id": deal_id})
        df.to_csv("hubspot_contacts_to_deals.csv", encoding="latin-1", index=False, errors='ignore')
        assoc_results = []
        for result in itterator:
            try:
                lists_to_append = result["associations"]["companies"]["results"]
                to_append = []
                for i in lists_to_append:
                    dict_to_append = {}
                    dict_to_append["contact_id"] = result["id"]
                    dict_to_append["company_id"] = i["id"]
                    to_append.append(dict_to_append)
                assoc_results.append(to_append)
            except Exception as e:
                assoc_results.append([{"contact_id": result["id"], "company_id": None}])
        contacts_id = []
        companies_id = []
        for index, item in enumerate(assoc_results):
            i = 0
            for a, b in item:
                contacts_id.append(item[i][a])
                companies_id.append(item[i][b])
                i += 1
        df = pd.DataFrame({"contact_id": contacts_id, "company_id": companies_id})
        df.to_csv("hubspot_contacts_to_companies.csv", encoding="latin-1", index=False, errors='ignore')

    def raw_export_companies(self, properties_dict):
        # Parameters
        results = []
        after_results = []
        previous_after = None
        after = 0

        while str(after).isnumeric():
            try:
                api_response = self.client.crm.companies.basic_api.get_page(
                    limit=100,
                    after=after,
                    properties=list(properties_dict.keys()),
                    associations=["deals"],
                    archived=False,
                )
                api_response = api_response.to_dict()
                results.extend(api_response['results'])
                print("Hubspot Companies Export has gathered " + str(len(results)) + " Companies")
                try:
                    after = api_response['paging']['next']['after']
                except:
                    after = api_response['paging']
            except ApiException as e:
                print("Exception when calling basic_api->get_page: %s\n" % e)
        property_results = []
        for result in results:
            property_results.append(result["properties"])
        df = pd.DataFrame(property_results)
        df.to_csv("raw_hubspot_companies.csv", encoding="latin-1", index=False, errors='ignore')
        assoc_results = []
        itterator = results.copy()
        for result in itterator:
            if result["associations"] is not None:
                lists_to_append = result["associations"]["deals"]["results"]
                to_append = []
                for i in lists_to_append:
                    dict_to_append = {}
                    dict_to_append["company_id"] = result["id"]
                    dict_to_append["deal_id"] = i["id"]
                    to_append.append(dict_to_append)
                assoc_results.append(to_append)
            else:
                assoc_results.append([{"company_id": result["id"], "deal_id": None}])
        company_id = []
        deal_id = []
        for index, item in enumerate(assoc_results):
            i = 0
            for a, b in item:
                company_id.append(item[i][a])
                deal_id.append(item[i][b])
                i += 1
        df = pd.DataFrame({"company_id": company_id, "deals_id": deal_id})
        df.to_csv("hubspot_companies_to_deals.csv", encoding="latin-1", index=False, errors='ignore')

    def raw_export_deals(self, properties_dict):
        # Assistant Variables
        results = []
        after_results = []
        previous_after = None
        after = 0
        while str(after).isnumeric():
            try:
                api_response = self.client.crm.deals.basic_api.get_page(
                    limit=100,
                    after=after,
                    properties=list(properties_dict.keys()),
                    archived=False,
                )
                api_response = api_response.to_dict()
                results.extend(api_response['results'])
                print("Hubspot Deals Export has gathered " + str(len(results)) + " Deals")
                try:
                    after = api_response['paging']['next']['after']
                except:
                    after = api_response['paging']
            except ApiException as e:
                print("Exception when calling basic_api->get_page: %s\n" % e)
        property_results = []
        for result in results:
            property_results.append(result["properties"])
        df = pd.DataFrame(property_results)
        df.to_csv("raw_hubspot_deals.csv", encoding="latin-1", index=False, errors='ignore')

    def export_emailing(self):
        # Assistant Variables
        vidOffset = 0 
        with open("refresh_token.txt", "r") as f:
            refresh_token = f.readline().split("\n")[0]
        req = requests.post(
            "https://api.hubapi.com/oauth/v1/token",
            headers={"Content-Type": "application/x-www-form-urlencoded;charset=utf-8"},
            data="grant_type=refresh_token&client_id=" + client_id + "&client_secret=" + client_secret + "&refresh_token=" + refresh_token)
        with open("refresh_token.txt", "w") as f:
            response = json.loads(req.text)
            response["refresh_token"]
            f.write(response["refresh_token"])
        headers = {"Authorization": "Bearer " + json.loads(req.text)["access_token"]}
        request = "https://api.hubapi.com/marketing-emails/v1/emails/with-statistics?"
        parameters = [("limit", 300), ("offset", vidOffset)]
        parametersParsed = urllib.parse.urlencode(parameters)
        requestURL = request + parametersParsed
        response = requests.get(url=requestURL, headers=headers)
        response_dict = json.loads(response.text)
        dictList = []
        for i in response_dict["objects"]:
            if i["isPublished"] == True:
                dictList.append(i)
        for index, i in enumerate(dictList):
            dictList[index] = {"id": i["id"],
                               "name": i["name"],
                               "create_date": i["created"],
                               "sent_date": i["publishDate"],
                               "subscription_name": i["subscriptionName"]
                               }
            try:
                dictList[index].update(i["stats"]["counters"])
            except:
                dictList[index].update({
                    'sent': "",
                    'open': "",
                    'delivered': "",
                    'bounce': "",
                    'unsubscribed': "",
                    'click': "",
                    'reply': "",
                    'dropped': "",
                    'selected': "",
                    'spamreport': "",
                    'suppressed': "",
                    'hardbounced': "",
                    'softbounced': "",
                    'pending': "",
                    'contactslost': "",
                    'notsent': ""})
        df = pd.DataFrame(dictList)
        for date_column in ["create_date", "sent_date"]:
            df[date_column] = pd.to_datetime(df[df[date_column].notna()][date_column], unit='ms', errors='raise')
            df[date_column] = df[date_column].dt.strftime('%Y-%m-%d')
        df.to_csv("hubspot_emailing.csv", encoding="latin-1", index=False, errors='ignore')

    def handle_raw_hubspot(self, csv_file, general_values, properties_dict, values_dict, date_columns, dtype={}):
        # Csv into Dataframe
        df = pd.read_csv(csv_file + ".csv", encoding="latin-1")
        # Rename Columns
        df.rename(columns=properties_dict, inplace=True)
        # Date Standarization
        date_columns = [char.lower() for char in date_columns]
        date_columns = ["_".join(char.split(" ")) for char in date_columns]
        # for date_column in date_columns:
        #     df[date_column] = pd.to_datetime(df[df[date_column].notna()][date_column], errors='ignore')
        #     df[date_column] = df[date_column].dt.strftime('%Y-%m-%d %H:%M:%S')
        # Adjust Datatypes
        for col in list(df.columns):
            if col not in date_columns:
                df[col] = df[col].astype(dtype[col], errors='ignore')
        # Rename Values
        values_dict.update(general_values)
        df = df.applymap(lambda x: str(x) if pd.notnull(x) else '')
        df.replace(values_dict, inplace=True)
        # Export
        df.to_csv(csv_file[4:] + ".csv", encoding="latin-1", index=False, errors='ignore')

    def export_meetings(self):
        print("Exporting Hubspot meetings")
        results = []
        after = 0
        while str(after).isnumeric() and len(results) < self.max_results:
            try:
                api_response = self.client.crm.objects.meetings.basic_api.get_page(
                    limit=100,
                    after=after,
                    archived=False,
                    properties=["hs_meeting_outcome",
                                "hubspot_owner_id",
                                "hs_meeting_start_time",
                                "hs_meeting_end_time"]
                )
                api_response = api_response.to_dict()
                results.extend(api_response['results'])
                print("Hubspot Meetings Export has gathered " + str(len(results)) + " Meetings")
                try:
                    after = api_response['paging']['next']['after']
                except:
                    after = api_response['paging']
            except ApiException:
                pass
        property_results = []
        for result in results:
            property_results.append(result["properties"])
        df = pd.DataFrame(property_results)
        df.to_csv("hubspot_meetings.csv", encoding="latin-1", index=False, errors='ignore')
        print("Hubspot meetings export finished")

    def associate_meetings_to_contacts(self):
        print("Associating Hubspot meetings to contacts")
        df = pd.read_csv("hubspot_meetings.csv")
        df = df[df["hs_meeting_outcome"].notnull()]
        meeting_ids = list(df.hs_object_id)
        meetings_to_contacts = {}
        counter = 0
        for i in meeting_ids:
            print(str(counter), "of", str(len(meeting_ids)), "Hubspot meetings associations")
            counter += 1
            try:
                api_response = self.client.crm.objects.meetings.basic_api.get_by_id(
                    meeting_id=i,
                    associations=["contacts", "companies"],
                    archived=False,
                )
                api_response = api_response.to_dict()
                try:
                    meetings_to_contacts[i] = api_response["associations"]["contacts"]["results"][0]["id"]
                except:
                    pass
            except ApiException:
                pass
        df_meetings_to_contacts = {'hs_object_id': meetings_to_contacts.keys(),
                                   'hs_contact_id': meetings_to_contacts.values()}
        df2 = pd.DataFrame(df_meetings_to_contacts)
        df2.to_csv("hubspot_meetings_to_contacts.csv", encoding="latin-1", index=False, errors='ignore')
        print("Hubspot contacts to meetings association finished")


class OdooToHubspot:
    def __init__(self):
        # API KEY
        self.access_token = ACCESS_TOKEN
        self.client = hubspot.Client.create(access_token = self.access_token)
        self.engine = create_engine(DATABASE_CONNECTION_URI)
    
    def clean_list(self, list_to_clean, list_to_compare):
        temp_list = []
        for item in list_to_clean:
            add = True
            for check_item in list_to_compare:
                if item == check_item:
                    add = False
            if add == True:
                temp_list.append(item)
        return temp_list

    def update_key_admins(self):
        engine = self.engine
        df_odoo_projects = pd.read_sql_table("odoo_projects", engine)
        df_hubspot_contacts = pd.read_sql_table("hubspot_contacts", engine)
        list_of_mails = df_odoo_projects[df_odoo_projects['mail_de_focal_point'].astype(str).apply(validate_email)]["mail_de_focal_point"].tolist()
        # To Lower Case
        list_of_mails = [list_of_mails[i].lower() for i in range(len(list_of_mails))]
        # Eliminating Duplicates
        temp_list = []
        [temp_list.append(x) for x in list_of_mails if x not in temp_list]
        list_of_mails = temp_list
        # Collecting Hubspot Contacts ID to Update
        list_of_mails_hubspot = df_hubspot_contacts[df_hubspot_contacts['email'].isin(list_of_mails)]["email"].tolist()
        # To Lower Case
        list_of_mails_hubspot = [list_of_mails_hubspot[i].lower() for i in range(len(list_of_mails_hubspot))]
        # Eliminating Duplicates between lists
        list_of_mails = self.clean_list(list_of_mails, list_of_mails_hubspot)
        update_contact_id_list = df_hubspot_contacts[
            (df_hubspot_contacts['email'].isin(list_of_mails)) & (df_hubspot_contacts["key_admin"] == "true")][
            "contact_id"].astype("int")
        # Update
        client = hubspot.Client.create(access_token = ACCESS_TOKEN)
        properties = {
            "key_admin": "true",
        }
        simple_public_object_input = SimplePublicObjectInput(properties=properties)
        for contact_id in update_contact_id_list:
            try:
                api_response = client.crm.contacts.basic_api.update(contact_id=contact_id, simple_public_object_input=simple_public_object_input)
                print(api_response)
            except ApiException as e:
                print("Exception when calling basic_api->create: %s\n" % e)
        for mail in list_of_mails:
            properties = {
                "email": mail,
                "key_admin": "true"
            }
            simple_public_object_input = SimplePublicObjectInput(properties=properties)
            try:
                api_response = client.crm.contacts.basic_api.create(simple_public_object_input=simple_public_object_input)
            except ApiException as e:
                print("Exception when calling basic_api->create: %s\n" % e)

    def odoo_companies_products(self):
        engine = self.engine
        query = """SELECT odoo_subscriptions_lines.company_name, odoo_subscriptions_lines.producto, odoo_entities.hubspot_company_id
        FROM odoo_subscriptions_lines
        LEFT JOIN odoo_entities ON odoo_subscriptions_lines.company_name = odoo_entities.name
        WHERE odoo_subscriptions_lines.etapa IN ('Suscripcion Activa','Suscripción Activa Parcial')"""
        external_data = {'[DQUS-R-C/MANT] Soporte y Actualizaciones debQ': ['debQ'], '[DQUS-R-A] debQ Core x Usuario Alquiler': ['debQ'], '[DQAP] debQ Citas Online x Sucursal': ['debQ', 'Citas'], '[DQVQ] debQ Fila Virtual x Sucursal': ['debQ', 'Fila Virtual'], '[DQSC-R-A] debQ Core x Sucursal Alquiler (hasta 15 usuarios)': ['debQ'], '[DQSR] debQ Encuestas de Satisfacción x Sucursal': ['debQ', 'Encuestas'], '[DQUS-R-C/MANT-M] Soporte y Actualizaciones debQ Core - Mensual': ['debQ'], '[DQUS-R-C/MANT] Soporte y Actualizaciones debQ Core': ['debQ'], '[DQAP] debQ Citas Online': ['debQ', 'Citas'], '[DQAP] debQ Citas Online x Sucursal Alquiler': ['debQ', 'Citas'], '[DQSR] debQ Encuestas de Satisfacción x Sucursal Alquiler': ['debQ', 'Encuestas'], '[DQVQ] debQ Fila Virtual x Sucursal Alquiler': ['debQ', 'Fila Virtual'], '[DSUS-R-A] debSign Cartelería Digital x Display Alquiler': ['debSign'], '[DSUS-R] debSign Cartelería Digital': ['debSign'], 'Rentokil': [''], '[DQSMS] Paquete SMS x mes': [''], '[DQRMD] debQ SMS Reminder x Sucursal': ['debQ', 'Reminder'], '[DQUS-R-C] debQ Core x Usuario Compra': ['debQ'], '[DSUS-R-C/MANT] Soporte y Actualizaciones debSign': ['debSign'], 'Soporte y Actualizaciones debQ Core - Mensual': ['debQ'], 'Facturado como debTouch': [''], 'AÑO 1': [''], 'OC': [''], '[DQVL] debQ Videollamada x Usuario Aquiler': ['debQ', 'Videollamada'], 'debSign Cartelería Digital x Display Alquiler': ['debSign'], '[DQRMD] debQ SMS Reminder x Sucursal Alquiler': ['debQ', 'Reminder'], 'debQ Recepcionesta': ['debQ'], '[DQAP-C/MANT] Soporte y Actualizaciones debQ Citas Online': ['debQ', 'Citas'], '[DQVQ-C/MANT] Soporte y Actualizaciones debQ Fila Virtual': ['debQ', 'Fila Virtual'], '[MN-HW-C] Mantenimiento de Hardware por punto en Zona Centro': ['Mantenimiento de Hardware'], '[DTKS-TK-R-I] Kiosk Robusto Renting con impresora de tickets': ['Renting HW'], '[MN-HW-R] Mantenimiento de Hardware por punto en Zona Región': ['Mantenimiento de Hardware'], '[DQUS-R-C/MANT-M] Soporte y Actualizaciones debQ Core - Mensual ': ['debQ'], '[DSUS-R-C] debSign Cartelería Digital x Display Compra': ['debSign'], '[DQUS-R-C/MANT] Soporte y Actualizaciones debQ Core - Anual': ['debQ'], '[DQUS-R-C/MANT-A] Soporte y Actualizaciones debQ Core - Anual': ['debQ'], '[DSUS-R-C/MANT-A] Soporte y Actualizaciones debSign - Anual': ['debSign'], 'Soporte de DESARROLLO': [''], '[DTKS-LT-R-I] All in One Kiosk Renting con impresora de tickets': ['Renting'], '[DQVQ-C/MANT-A] Soporte y Actualizaciones debQ Fila Virtual - Anual': ['debQ', 'Fila Virtual'], '[DQAP-C/MANT-A] Soporte y Actualizaciones debQ Citas Online - Anual': ['debQ', 'Citas'], 'debQ Citas Online x Sucursal Alquiler': ['debQ', 'Citas'], '[DQAP-C/MANT-M] Soporte y Actualizaciones debQ Citas Online - Mensual': ['debQ', 'Citas'], 'debQ Encuestas de Satisfacción x Sucursal Alquiler': ['debQ', 'Encuestas'], 'debQ Core x Sucursal Alquiler (hasta 15 usuarios)': ['debQ'], 'debQ Videollamada x Usuario Aquiler': ['debQ', 'Videollamada'], '[DQLT-CITAS/FV] debQ Citas Web + Fila Virtual + debQ Lite': ['debQ', 'Citas', 'Fila Virtual'], '[DSUS-R-C/MANT-M] Soporte y Actualizaciones debSign - Mensual': ['debSign'], '[DQLT-CITAS] debQ Citas Web + debQ Lite': ['debQ', 'Citas'], '[DQEMAIL] Paquete EMAIL x mes': [''], 'Preatendedor Alquiler x Compañía': ['Preatendedor'], 'debQ Core x Usuario Alquiler': ['debQ'], 'debQ Fila Virtual x Sucursal Alquiler': ['debQ', 'Fila Virtual'], 'Servicio prueba': [''], '[US-DEB-15] Chatbot Enterprise': ['Chatbot'], '[US-DEB-9] Citas Virtuales': ['Citas'], '[US-DEB-7] debSign Digital Signage': ['debSign'], '[US-DEB-8] Fila Virtual': ['Fila Virtual'], '[US-DEB-10] Encuestas': ['Encuestas'], '[US-DEB-17] Preatendedor Enterprise': ['Preatendedor'], '[US-DEB-11] Atención Virtual SMB': [''], '[US-DEB-13] Complemento Grabación de Atención por hora': [''], '[US-DEB-2] debQ Core Branch Paquete 15 Operadores': ['debQ'], '[US-DEB-3] debQ Core Branch Paquete 15 Operadores': ['debQ'], '[US-DEB-6] debQ Lite + Citas Virtuales + Fila Virtual + Customer Feedback': ['debQ', 'Citas', 'Fila Virtual'], '[US-DEB-1] debQ Core Branch Paquete 5 Operadores': ['debQ'], '[US-DEB-54] SMS Unitario': [''], '[US-DEB-5] debQ Core Operador Adicional': ['debQ'], '[US-DEB-4] debQ Core Branch Paquete 30 Operadores': ['debQ'], '[US-DEB-14] Chatbot SMB': ['Chatbot'], 'debQ Core Branch 1 Operado\nLorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.': ['debQ'], 'Atención Virtual 1 Operador': [''], '[DTKS-LT-A-I] All in One Kiosk Renting con impresora de tickets': ['Renting HW'], 'Producto prueba suscripción': [''], '[US-DEB-12] Hora Adicional Atención Virtual SMB': [''], 'Complemento Grabación de Atención por hora': [''], 'debQ 1 operador Videollamada': ['debQ', 'Videollamada']}
        for i in external_data:
            external_data[i] = ";".join(external_data[i])
        df = pd.read_sql_query(query, engine)
        df["producto"].replace(external_data, inplace=True)
        df_unique = pd.DataFrame(pd.unique(df["company_name"]))
        df_unique.rename(columns={0: "company_name"}, inplace=True)
        hub_ids = []
        for comp_df_un in df_unique.company_name:
            for comp_df, hub_id in zip(df.company_name, df.hubspot_company_id):
                if comp_df_un == comp_df:
                    hub_ids.append(hub_id)
                    break
        df_unique["hubspot_company_id"] = hub_ids
        df_dict = {"company": [],
                   "hubspot_company_id": [],
                   "productos": []}
        for company, hub_id in zip(df_unique.company_name, df_unique.hubspot_company_id):
            prod_list = []
            for prod in df[df.company_name == company].producto:
                prod_list.extend(prod.split(";"))
            df_dict["company"].append(company)
            df_dict["hubspot_company_id"].append(hub_id)
            df_dict["productos"].extend([prod_list])
        for enum, comp in enumerate(df_dict["company"]):
            df_dict["productos"][enum] = list(set(df_dict["productos"][enum]))
        new_df = pd.DataFrame.from_dict(df_dict)
        new_df["productos"] = new_df["productos"].apply(lambda x: [i for i in x if i in ["debQ", "debSign", "Citas", "Encuestas", "Fila Virtual", "Reminder", "Videollamada", "Mantenimiento de Hardware", "Renting HW", "Preatendedor", "Chatbot"]])
        new_df["productos"] = new_df["productos"].str.join(";")
        df_to_upload = new_df[["hubspot_company_id", "productos"]].copy()
        return df_to_upload

    def odoo_companies_relations(self):
        engine = self.engine
        query = """SELECT ent.hubspot_company_id, ent.relacion as odoo_relacion,
        SUM(CASE subs.etapa
            WHEN 'Suscripcion Activa' THEN 1
            WHEN 'Activas parciales' THEN 1
            ELSE 0
        END) AS suscripciones_activas,
        SUM(CASE subs.etapa
            WHEN 'Suscripción de Inicialización' THEN 1
            ELSE 0
        END) AS nuevas_suscripciones,
        SUM(CASE subs.etapa
            WHEN 'Suscripción Pausada' THEN 1
            ELSE 0
        END) AS suscripciones_pausadas,
        SUM(CASE subs.etapa
            WHEN 'Suscripcion de Baja' THEN 1
            WHEN 'Suscripción cancelada' THEN 1
            WHEN 'Suscripcion Vencida' THEN 1
            ELSE 0
        END) AS suscripciones_de_baja,
        SUM(CASE subs.etapa
            WHEN 'Demo Activa' THEN 1
            WHEN 'Demo en Proceso' THEN 1
            ELSE 0
        END) AS demos_activa,
        SUM(CASE subs.etapa
            WHEN 'Demo Cancelada' THEN 1
            WHEN 'Demo Vencida' THEN 1
            WHEN 'Demo Anulada' THEN 1
            ELSE 0
        END) AS demos_de_baja
        FROM odoo_subscriptions subs
        JOIN odoo_entities ent ON ent.name = subs.compania
        GROUP BY ent.hubspot_company_id, ent.relacion"""
            
        empty_list = []
        [empty_list.append("") for item in range(len(df))]
        df["relacion"] = empty_list
        df.loc[(df["odoo_relacion"] == "Partner") & (df["relacion"] == ""), "relacion"] = "Partner"
        df.loc[(df["suscripciones_activas"] > 0) & (df["relacion"] == "") & (df["odoo_relacion"] == "Cliente directo"), "relacion"] = "Cliente Directo"
        df.loc[(df["suscripciones_activas"] > 0) & (df["relacion"] == "") & (df["odoo_relacion"] == "Cliente Final"), "relacion"] = "Cliente Final"
        df.loc[(df["suscripciones_activas"] > 0) & (df["relacion"] == ""), "relacion"] = "Cliente"
        df.loc[(df["nuevas_suscripciones"] > 0) & (df["relacion"] == ""), "relacion"] = "Nuevo Cliente"
        df.loc[(df["suscripciones_pausadas"] > 0) & (df["relacion"] == ""), "relacion"] = "Cliente Pausado"
        df.loc[(df["demos_activa"] > 0) & (df["relacion"] == ""), "relacion"] = "Potencial Cliente"
        df.loc[(df["suscripciones_de_baja"] > 0) & (df["relacion"] == ""), "relacion"] = "Ex Cliente"
        df.loc[(df["demos_de_baja"] > 0) & (df["relacion"] == ""), "relacion"] = "Ex Potencial Cliente"
        df_to_upload = df[["hubspot_company_id", "relacion"]].copy()
        return(df_to_upload)

    def update_companies_properties(self, list_of_dfs):
        engine = self.engine
        query = """SELECT hubspot_company_id, name FROM odoo_entities WHERE hubspot_company_id != '' """
        final_df = pd.read_sql_query(query, engine)
        for df in list_of_dfs:
            final_df = final_df.merge(df, how="left", on="hubspot_company_id")
        final_df = final_df.fillna("")
        final_df = final_df[final_df["hubspot_company_id"].str.isnumeric()]
        final_df = final_df.drop(final_df[final_df["hubspot_company_id"].duplicated()].index)
        failed_companies = []
        for hub_id in final_df.hubspot_company_id:
            if str(hub_id).isnumeric():
                company_name = list(final_df.loc[final_df["hubspot_company_id"] == str(hub_id)]["name"])[0]
                properties = {"productos_adquiridos__api_": list(final_df[final_df["hubspot_company_id"] == str(hub_id)]["productos"])[0], "relacion_cliente__api_": list(final_df[final_df["hubspot_company_id"] == hub_id]["relacion"])[0]}
                simple_public_object_input = SimplePublicObjectInput(properties=properties)
                try:
                    api_response = self.client.crm.companies.basic_api.update(company_id=hub_id, simple_public_object_input=simple_public_object_input)
                    print(company_name + "'s products and relations updated in Hubspot")
                except:
                    print("Error in  " + company_name + "'s products and relations update")
                    failed_companies.append(company_name)
            else:
                failed_companies.append(company_name)
        print("There was a failure updating the following companie's products and relations:")
        print(failed_companies)

In [23]:
class SqlAPI:

    def __init__(self):
        self.conn = create_engine(DATABASE_CONNECTION_URI)

    def str_type_into_sqltype(self, datatype_dict):
        for i in datatype_dict:
            if datatype_dict[i] == "string":
                datatype_dict[i] = types.TEXT()
            elif datatype_dict[i] == "int64":
                datatype_dict[i] = types.NUMERIC()
            elif datatype_dict[i] == "float64":
                datatype_dict[i] = types.FLOAT()
            elif datatype_dict[i] == "datetime64":
                datatype_dict[i] = types.TIMESTAMP()
            elif datatype_dict[i] == "bool":
                datatype_dict[i] = types.BOOLEAN()
        return datatype_dict

    def insert_df(self, dataframe, table, dtype, index=False, if_exists="replace"):
        dataframe.to_sql(
            name=table,
            con=self.conn,
            index=index,
            if_exists=if_exists,
            method='multi',
            dtype=dtype,
            chunksize=10000
        )

    def update_table(self, table, dtype):

        print("Update Started " + table)
        csv_name = table + ".csv"
        df = pd.read_csv(csv_name, encoding='latin-1')
        columns = [column.lower() for column in df.columns]
        columns = ["_".join(column.split(" ")) for column in columns]
        df.columns = columns
        dtype = self.str_type_into_sqltype(dtype)
        self.insert_df(df, table, dtype)
        print("Update Finished " + table)

In [24]:
def dict_into_simple_dict(complex_dict, simple_value):
    headers = list(complex_dict.keys())
    simple_values_list = []
    for k in complex_dict:
        simple_values_list.append(complex_dict[k][simple_value])
    return dict(zip(headers, simple_values_list))


def list_of_datetimes(complex_dict):
    lst = []
    simple_dict = dict_into_simple_dict(complex_dict, "datatype")
    for item in simple_dict:
        if simple_dict[item] == "datetime64":
            lst.append(item)
    return lst

def update_database():
    # DB Schema Handling
    api_schema = json.load(open("api_schema.json"))["API"]
    hubspot_schema = api_schema["Hubspot"]
   
    # Hubspot Schema
    h_contacts_properties = dict_into_simple_dict(hubspot_schema["Contacts"]["Properties"], "header_name")
    h_contacts_values = hubspot_schema["Contacts"]["Values"]
    h_contacts_datecolumns = list_of_datetimes(hubspot_schema["Contacts"]["Properties"])
    h_contacts_datecolumns = [h_contacts_properties[i] for i in h_contacts_datecolumns]
    h_contacts_datatypes = dict(zip(h_contacts_properties.values(),
                                    dict_into_simple_dict(hubspot_schema["Contacts"]["Properties"],
                                                          "datatype").values()))
    h_companies_properties = dict_into_simple_dict(hubspot_schema["Companies"]["Properties"], "header_name")
    h_companies_values = hubspot_schema["Companies"]["Values"]
    h_companies_datecolumns = list_of_datetimes(hubspot_schema["Companies"]["Properties"])
    h_companies_datecolumns = [h_companies_properties[i] for i in h_companies_datecolumns]
    h_companies_datatypes = dict(zip(h_companies_properties.values(),
                                     dict_into_simple_dict(hubspot_schema["Companies"]["Properties"],
                                                           "datatype").values()))
    h_deals_properties = dict_into_simple_dict(hubspot_schema["Deals"]["Properties"], "header_name")
    h_deals_values = hubspot_schema["Deals"]["Values"]
    h_deals_datecolumns = list_of_datetimes(hubspot_schema["Deals"]["Properties"])
    h_deals_datecolumns = [h_deals_properties[i] for i in h_deals_datecolumns]
    h_deals_datatypes = dict(
        zip(h_deals_properties.values(),
            dict_into_simple_dict(hubspot_schema["Deals"]["Properties"], "datatype").values()))
    h_generalvalues = hubspot_schema["General Values"]
    
    # Class Variables Declaration
    s = SqlAPI()
    h = HubspotAPI()

    # Hubspot Export
    # h.raw_export_contacts(h_contacts_properties)
    # h.raw_export_companies(h_companies_properties)
    h.raw_export_deals(h_deals_properties)

    # try:
    #     h.export_emailing()
    # except:
    #     "Hubspot Emailing export has failed"

    # try:
    #     h.export_meetings()
    # except:
    #     print("Hubspot Meetings export has failed")
    # try:
    #     h.associate_meetings_to_contacts()
    # except:
    #     print("Hubspot Meetings associations has failed")

    # Data Handle
    # h.handle_raw_hubspot("raw_hubspot_contacts", h_generalvalues, h_contacts_properties, h_contacts_values,
    #                      h_contacts_datecolumns, h_contacts_datatypes)
    # h.handle_raw_hubspot("raw_hubspot_companies", h_generalvalues, h_companies_properties, h_companies_values,
    #                      h_companies_datecolumns, h_companies_datatypes)
    h.handle_raw_hubspot("raw_hubspot_deals", h_generalvalues, h_deals_properties, h_deals_values, h_deals_datecolumns,
                         h_deals_datatypes)
  
    # Tablas a actualizar en la DB
    tables = [
        # "hubspot_contacts",
        # "hubspot_companies",
        "hubspot_deals",
        # "hubspot_contacts_to_companies",
        # "hubspot_contacts_to_deals",
        # "hubspot_companies_to_deals",
        #"hubspot_meetings_to_contacts",
        #"hubspot_meetings",
        # "hubspot_emailing"
    ]

    tables_datatypes = [
        h_contacts_datatypes,
        h_companies_datatypes,
        h_deals_datatypes,
        {},
        {},
        {},
        {},
        {},
        {}
    ]
    for index, table in enumerate(tables):
        s.update_table(table, tables_datatypes[index])

In [25]:
update_database()

# os.remove("raw_hubspot_companies" + ".csv")
# os.remove("raw_hubspot_contacts" + ".csv")
os.remove("raw_hubspot_deals" + ".csv")
# os.remove("hubspot_companies_to_deals" + ".csv")
# os.remove("hubspot_companies" + ".csv")
# os.remove("hubspot_contacts_to_companies" + ".csv")
# os.remove("hubspot_contacts_to_deals" + ".csv")
# os.remove("hubspot_contacts" + ".csv")
# os.remove("hubspot_deals" + ".csv")
#os.remove("hubspot_meetings_to_contacts" + ".csv")
#os.remove("hubspot_meetings" + ".csv")
# os.remove("hubspot_emailing" + ".csv")

Hubspot Deals Export has gathered 100 Deals
Hubspot Deals Export has gathered 200 Deals
Hubspot Deals Export has gathered 300 Deals
Hubspot Deals Export has gathered 400 Deals
Hubspot Deals Export has gathered 500 Deals
Hubspot Deals Export has gathered 600 Deals
Hubspot Deals Export has gathered 700 Deals
Hubspot Deals Export has gathered 800 Deals
Hubspot Deals Export has gathered 900 Deals
Hubspot Deals Export has gathered 1000 Deals
Hubspot Deals Export has gathered 1100 Deals
Hubspot Deals Export has gathered 1200 Deals
Hubspot Deals Export has gathered 1300 Deals
Hubspot Deals Export has gathered 1400 Deals
Hubspot Deals Export has gathered 1500 Deals
Hubspot Deals Export has gathered 1600 Deals
Hubspot Deals Export has gathered 1700 Deals
Hubspot Deals Export has gathered 1800 Deals
Hubspot Deals Export has gathered 1900 Deals
Hubspot Deals Export has gathered 2000 Deals
Hubspot Deals Export has gathered 2100 Deals
Hubspot Deals Export has gathered 2200 Deals
Hubspot Deals Expor

C:\Users\Magdalena\AppData\Local\Temp\ipykernel_15124\2132803704.py:242: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x) if pd.notnull(x) else '')


Update Started hubspot_deals
Update Finished hubspot_deals
